In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import librosa as lb
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Models

In [ ]:
hbnet=keras.models.load_model('../input/part-3-feature-extraction-modeling-95-acc/hbNet')
oneD=keras.models.load_model('../input/1dnet-for-rdd/1Dnet')
unTuned=keras.models.load_model('../input/hbnet-untuned-for-rdd//hbNet_unTuned/')

In [ ]:
hbnet.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer='nadam',
    metrics=['sparse_categorical_accuracy']
)

oneD.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer='nadam',
    metrics=['sparse_categorical_accuracy']
)

unTuned.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer='nadam',
    metrics=['sparse_categorical_accuracy']
)

# Loading and Preprocessing Data

In [ ]:
from sklearn.preprocessing import LabelEncoder

train=pd.read_csv('../input/part-2-handel-imbalance-creating-spectrogram/train.csv')
val=pd.read_csv('../input/part-2-handel-imbalance-creating-spectrogram/val.csv')
                  
ytrain=train.disease
yval=val.disease
                  
le=LabelEncoder()
ytrain=le.fit_transform(ytrain)
yval=le.transform(yval)

In [ ]:
def getFeatures_net(path):
    soundArr,sample_rate=lb.load(path)
    mfcc=lb.feature.mfcc(y=soundArr,sr=sample_rate)
    cstft=lb.feature.chroma_stft(y=soundArr,sr=sample_rate)
    mSpec=lb.feature.melspectrogram(y=soundArr,sr=sample_rate)

    return mfcc,cstft,mSpec

root='../input/part-1-preprocessing/processed_audio_files/'
mfcc,cstft,mSpec=[],[],[]

for idx,row in val.iterrows():
    path=root + row['filename']
    a,b,c=getFeatures_net(path)
    mfcc.append(a)
    cstft.append(b)
    mSpec.append(c)
    
mfcc_val=np.array(mfcc)
cstft_val=np.array(cstft)
mSpec_val=np.array(mSpec)

In [ ]:
def getFeatures_oneD(path):
    soundArr,sample_rate=lb.load(path)
    mfcc=np.mean(lb.feature.mfcc(y=soundArr,sr=sample_rate,n_mfcc=40),axis=0)
    cstft=np.mean(lb.feature.chroma_stft(y=soundArr,sr=sample_rate,n_chroma=24),axis=0)
    mSpec=np.mean(lb.feature.melspectrogram(y=soundArr,sr=sample_rate),axis=0)
    tonnetz=np.mean(lb.feature.tonnetz(y=soundArr,sr=sample_rate),axis=0)
    specCetroid=np.mean(lb.feature.spectral_centroid(y=soundArr,sr=sample_rate),axis=0)

    return [mfcc,cstft,mSpec,tonnetz,specCetroid]

root='../input/part-1-preprocessing/processed_audio_files/'
valFeatures=[]
i=0
for idx,row in val.iterrows():
    path=root + row['filename']
    X=np.array(getFeatures_oneD(path))
    valFeatures.append(X.T)

valFeatures=np.array(valFeatures)

# Evaluating Models

In [ ]:
hbnet.evaluate({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val},yval)

In [ ]:
oneD.evaluate(valFeatures,yval)

In [ ]:
unTuned.evaluate({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val},yval)

# Comapring Metrics

In [ ]:
yprob_hbnet=hbnet.predict({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val})
ytrue_hbnet = yprob_hbnet.argmax(axis=-1)

In [ ]:
ytrue_oneD=oneD.predict_classes(valFeatures)
yprob_oneD=oneD.predict_proba(valFeatures)

In [ ]:
yprob_unTuned=unTuned.predict({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val})
ytrue_unTuned = yprob_unTuned.argmax(axis=-1)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

* hbNet

In [ ]:
print('Precision: ',precision_score(yval,ytrue_hbnet,average='weighted'))
print('Recall:    ',recall_score(yval,ytrue_hbnet,average='weighted'))
print('F1 Score:  ',f1_score(yval,ytrue_hbnet,average='weighted'))

* oneD

In [ ]:
print('Precision: ',precision_score(yval,ytrue_oneD,average='weighted'))
print('Recall:    ',recall_score(yval,ytrue_oneD,average='weighted'))
print('F1 Score:  ',f1_score(yval,ytrue_oneD,average='weighted'))

* unTuned

In [ ]:
print('Precision: ',precision_score(yval,ytrue_unTuned,average='weighted'))
print('Recall:    ',recall_score(yval,ytrue_unTuned,average='weighted'))
print('F1 Score:  ',f1_score(yval,ytrue_unTuned,average='weighted'))

**Reciver Operator Characterstics**

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.preprocessing import label_binarize

In [ ]:
yval_b = label_binarize(yval, classes=[0, 1, 2, 3, 4, 5, 6, 7])

* hbnet

In [ ]:
n_classes=8
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(yval_b[:, i], yprob_hbnet[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
lw = 2
figure(num=None, figsize=(15, 6), dpi=100, facecolor='w', edgecolor='k')
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],lw=lw, label='ROC (class= %d) (area = %0.2f)' % (i,roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operator Characteristics')
plt.legend(loc="lower right")
plt.show()

* oneD

In [ ]:
n_classes=8
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(yval_b[:, i], yprob_oneD[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
lw = 2
figure(num=None, figsize=(15, 6), dpi=100, facecolor='w', edgecolor='k')
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],lw=lw, label='ROC (class= %d) (area = %0.2f)' % (i,roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operator Characteristics')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes=8
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(yval_b[:, i], yprob_unTuned[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
lw = 2
figure(num=None, figsize=(15, 6), dpi=100, facecolor='w', edgecolor='k')
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],lw=lw, label='ROC (class= %d) (area = %0.2f)' % (i,roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operator Characteristics')
plt.legend(loc="lower right")
plt.show()